# Projekt 2: Klassifikation mit kategorischen Daten

In [7]:
from sklearn.datasets import fetch_openml

In [8]:
data = fetch_openml(data_id=1590, parser="auto")

## Datensatz explorieren

1. Welche Features enthält der Datensatz? Was repräsentiert das Label?

In [9]:
data

{'data':        age     workclass  fnlwgt     education  education-num  \
 0       25       Private  226802          11th              7   
 1       38       Private   89814       HS-grad              9   
 2       28     Local-gov  336951    Assoc-acdm             12   
 3       44       Private  160323  Some-college             10   
 4       18           NaN  103497  Some-college             10   
 ...    ...           ...     ...           ...            ...   
 48837   27       Private  257302    Assoc-acdm             12   
 48838   40       Private  154374       HS-grad              9   
 48839   58       Private  151910       HS-grad              9   
 48840   22       Private  201490       HS-grad              9   
 48841   52  Self-emp-inc  287927       HS-grad              9   
 
            marital-status         occupation relationship   race     sex  \
 0           Never-married  Machine-op-inspct    Own-child  Black    Male   
 1      Married-civ-spouse    Farming-fishin

In [10]:
data.feature_names

['age',
 'workclass',
 'fnlwgt',
 'education',
 'education-num',
 'marital-status',
 'occupation',
 'relationship',
 'race',
 'sex',
 'capital-gain',
 'capital-loss',
 'hours-per-week',
 'native-country']

In [11]:
data.target_names

['class']

In [12]:
data.target

0        <=50K
1        <=50K
2         >50K
3         >50K
4        <=50K
         ...  
48837    <=50K
48838     >50K
48839    <=50K
48840    <=50K
48841     >50K
Name: class, Length: 48842, dtype: category
Categories (2, object): ['<=50K', '>50K']

2. Erstelle eine Liste `categorical_features`, die die Namen aller kategorischen Features enthält und eine Liste `numerical_features`, die alle numerischen Features enthält.

Tipp: `numerical_features` muss alle Features aus `data.feature_names` enthalten, die nicht schon in `categorical_features` enthalten sind.

In [13]:
categorical_features = ["workclass",
"education",
"marital-status",
"occupation",
"relationship",
"race",
"sex",
"native-country"]

In [14]:
numerical_features = [feature for feature in data.feature_names if feature not in categorical_features]

In [15]:
numerical_features

['age',
 'fnlwgt',
 'education-num',
 'capital-gain',
 'capital-loss',
 'hours-per-week']

In [16]:
len(categorical_features) + len(numerical_features) == len(data.feature_names)

True

3. Splitte die Daten in einen Test- und einen Trainings-Datensatz

In [17]:
from sklearn.model_selection import train_test_split

In [18]:
x_train, x_test, y_train, y_test = train_test_split(data.data, data.target)

## Preprocessing aufbauen

4. Die Label müssen in ein binäres Format, also auf `0` (`<=50k`) und `1` (`>50k`), konvertiert werden. Dazu kann der `OrdinalEncoder` eingesetzt werden, um sowohl `y_train` als auch `y_test` entsprechend zu konvertieren.

Erzeuge zunächst eine Liste der möglichen Werte in `y`, die mit `categories` an den `OrdinalEncoder` übergeben werden kann (Dimensionen beachten!).

Tipp: Mit `y_train.values` kann auf ein Numpy-Array zugegriffen werden.

In [19]:
target_values = data.target.values.unique()

In [20]:
from sklearn.preprocessing import OrdinalEncoder

In [21]:
label_encoder = OrdinalEncoder(categories=[target_values], handle_unknown="error")

In [22]:
label_encoder.fit(y_train.values.reshape(-1, 1))

OrdinalEncoder(categories=[['<=50K', '>50K']
Categories (2, object): ['<=50K', '>50K']])

In [23]:
label_encoder.categories_

[array(['<=50K', '>50K'], dtype=object)]

In [24]:
y_train = label_encoder.transform(y_train.values.reshape(-1, 1))

In [25]:
y_test = label_encoder.transform(y_test.values.reshape(-1, 1))

5. Die kategorischen Features sollen One-Hot kodiert werden. Dazu kann der `OneHotEncoder` aus `sklearn.preprocessing` eingesetzt werden. Werte für die Features, die im Trainings-Set nicht vorkommen, sollen ignoriert werden.

Der Encoder für die kategorischen Features soll in diesem Schritt zunächst nur definiert werden, wir werden ihn später gemeinsam mit dem numerischen Encoder und dem Modell fitten.

In [26]:
from sklearn.preprocessing import OneHotEncoder

In [27]:
categorical_transformer = OneHotEncoder(handle_unknown="ignore")

6. Für die kategorischen Features müssen wir bereits eine Pipeline aufbauen, denn einige Spalten enthalten unbekannte Werte.

- Überprüfe mit `x_train.isna().any()`, welche Spalten betroffen sind
- Wähle anhand der [Dokumentation für `SimpleImputer](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html) eine geeignete Strategie
- Kombiniere `SimpleImputer` und einen `OneHotEncoder` in einer `Pipeline`.



In [28]:
x_train.isna().any()

age               False
workclass          True
fnlwgt            False
education         False
education-num     False
marital-status    False
occupation         True
relationship      False
race              False
sex               False
capital-gain      False
capital-loss      False
hours-per-week    False
native-country     True
dtype: bool

In [29]:
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

In [30]:
categorical_transformer = Pipeline(
    steps=[("imputer", SimpleImputer(strategy="most_frequent")), ("scaler", OneHotEncoder(handle_unknown="ignore"))]
)

6. Mit dem `ColumnTransformer` können wir die Pipelines für numerische und kategorische Features nun zusammensetzen.

Fitte testweise den `preprocessor` auf dem Trainingsdatensatz und konvertiere den Testdatensatz. Vergleiche die Dimensionen von `x_test` und dem transformierten Datensatz. Was fällt dir auf?

In [31]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler

In [32]:
preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numerical_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)

In [33]:
preprocessor.fit(x_train)

ColumnTransformer(transformers=[('num', StandardScaler(),
                                 ['age', 'fnlwgt', 'education-num',
                                  'capital-gain', 'capital-loss',
                                  'hours-per-week']),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('scaler',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['workclass', 'education', 'marital-status',
                                  'occupation', 'relationship', 'race', 'sex',
                                  'native-country'])])

In [34]:
preprocessor.transform(x_test)

<12211x105 sparse matrix of type '<class 'numpy.float64'>'
	with 170954 stored elements in Compressed Sparse Row format>

## Pipeline aufbauen

7. Baue eine `Pipeline` bestehend aus zwei Schritten: Dem `preprocessor` und einem `RandomForestClassifier`.

Fitte diese testweise auf dem Trainings-Datensatz und evaluiere sie auf dem Test-Datensatz.

In [35]:
from sklearn.ensemble import RandomForestClassifier

In [36]:
model = Pipeline(
    steps=[("preprocessor", preprocessor), ('rfc', RandomForestClassifier())]
)
model.fit(x_train, y_train)

/home/kilian/.pyenv/versions/3.9/envs/ml-mit-scikit-learn/lib/python3.10/site-packages/sklearn/base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['age', 'fnlwgt',
                                                   'education-num',
                                                   'capital-gain',
                                                   'capital-loss',
                                                   'hours-per-week']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('scaler',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['workclass', 'education',
                                                   'marital-status',
                                                   'occupation', 'relationship',
                                                   'race', 'sex',
                                                   'native-country'])])),
                ('rfc', RandomForestClassifier())])

In [37]:
model.score(x_test, y_test)

0.856850380804193

## Grid Search mit Cross Validation

8. Um die optimalen Parameter zu finden, setzen wir `GridSearchCV` ein, mit der Grid-Search und Cross-Validation kombiniert werden.

Lege einen sinnvollen Parameter-Raum fest und führe einen Fit der kompletten Modell-Pipeline durch.

Was ist die beste Parameter-Kombination? Lässt sich noch eine bessere Kombination finden?

Tipp: Es ist in aller Regel hilfreich, den Einfluss einzelner Parameter zumindest qualitativ zu untersuchen.

In [38]:
from sklearn.model_selection import GridSearchCV

parameter_grid = {"rfc__max_depth": [5, 10], "rfc__n_estimators": [8]}
search = GridSearchCV(model, parameter_grid)
search.fit(x_train, y_train.ravel())

GridSearchCV(estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         StandardScaler(),
                                                                         ['age',
                                                                          'fnlwgt',
                                                                          'education-num',
                                                                          'capital-gain',
                                                                          'capital-loss',
                                                                          'hours-per-week']),
                                                                        ('cat',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer(strategy='most_frequent')),
                                                                                         ('scaler',
                                                                                          OneHotEncoder(handle_unknown='ignore'))]),
                                                                         ['workclass',
                                                                          'education',
                                                                          'marital-status',
                                                                          'occupation',
                                                                          'relationship',
                                                                          'race',
                                                                          'sex',
                                                                          'native-country'])])),
                                       ('rfc', RandomForestClassifier())]),
             param_grid={'rfc__max_depth': [5, 10], 'rfc__n_estimators': [8]})

In [39]:
search.best_estimator_

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['age', 'fnlwgt',
                                                   'education-num',
                                                   'capital-gain',
                                                   'capital-loss',
                                                   'hours-per-week']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('scaler',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['workclass', 'education',
                                                   'marital-status',
                                                   'occupation', 'relationship',
                                                   'race', 'sex',
                                                   'native-country'])])),
                ('rfc', RandomForestClassifier(max_depth=10, n_estimators=8))])

In [40]:
search.best_estimator_.score(x_test, y_test)

0.8539022193104577